In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [ ]:
# Setup
np.random.seed(42)
periods = pd.date_range(start='2024-01-01', end='2025-12-01', freq='MS')
industries = ['Healthcare', 'Consumer Goods', 'Retail', 'Technology', 'F&B']
service_lines = ['Digital Strategy', 'Creative Production', 'Media Performance']
tiers = ['Tier 1', 'Tier 2', 'Tier 3']
account_managers = [f'AM-{i:02d}' for i in range(1, 11)]

In [9]:
# --- 2. GENERATE FACT_REVENUE & FACT_COSTS (Transaksi Bulanan) ---
rev_list = []
cost_list = []
active_status = {c['client_id']: True for c in clients_data}

for current_month in periods:
    for _, client in dim_clients.iterrows():
        c_id = client['client_id']
        
        # Logika: Hanya proses jika sudah kontrak dan belum churn
        if client['contract_start_date'] <= current_month and active_status[c_id]:
            
            # Setiap klien bisa ambil 1-2 jasa secara simultan
            n_services = np.random.choice([1, 2], p=[0.7, 0.3])
            services = np.random.choice(service_lines, n_services, replace=False)
            
            for svc in services:
                # Logika Revenue berdasarkan Tier
                if client['client_tier'] == 'Tier 1':
                    rev = np.random.randint(90, 220) * 1_000_000
                    base_hrs = np.random.randint(80, 150)
                elif client['client_tier'] == 'Tier 2':
                    rev = np.random.randint(35, 89) * 1_000_000
                    base_hrs = np.random.randint(35, 75)
                else:
                    rev = np.random.randint(12, 34) * 1_000_000
                    base_hrs = np.random.randint(12, 30)

                # --- PENANAMAN BIAS BISNIS (The "Audit" Target) ---
                ext_cost = 0
                # 1. Creative Production: Biaya vendor & jam kerja bengkak
                if svc == 'Creative Production':
                    ext_cost = rev * np.random.uniform(0.35, 0.65)
                    base_hrs *= 1.4
                # 2. Media Buying: High efficiency (low hours)
                if svc == 'Media Buying':
                    base_hrs *= 0.45
                # 3. Tier 1 Overhead: Klien besar biasanya butuh meeting lebih lama
                if client['client_tier'] == 'Tier 1':
                    base_hrs *= 1.25

                # Record Revenue
                rev_list.append({
                    'period': current_month,
                    'client_id': c_id,
                    'service_line': svc,
                    'monthly_revenue': int(rev)
                })
                
                # Record Costs
                cost_list.append({
                    'period': current_month,
                    'client_id': c_id,
                    'service_line': svc,
                    'man_hours_billed': int(base_hrs),
                    'external_production_cost': int(ext_cost),
                    'internal_cost_per_hour': 350000
                })
            
            # Simulasi Churn (Probabilitas 2% per bulan)
            if np.random.random() < 0.02:
                active_status[c_id] = False

In [10]:
# Konversi ke DataFrame
fact_revenue = pd.DataFrame(rev_list)
fact_costs = pd.DataFrame(cost_list)

In [11]:
# --- 3. EXPORT KE CSV ---
dim_clients.to_csv('../data/raw/dim_clients.csv', index=False)
fact_revenue.to_csv('../data/raw/fact_revenue.csv', index=False)
fact_costs.to_csv('../data/raw/fact_costs.csv', index=False)

print(f"File Berhasil Dibuat!")
print(f"dim_clients: {len(dim_clients)} baris")
print(f"fact_revenue: {len(fact_revenue)} baris")
print(f"fact_costs: {len(fact_costs)} baris")

File Berhasil Dibuat!
dim_clients: 150 baris
fact_revenue: 3594 baris
fact_costs: 3594 baris
